In [1]:
# 基本パラメータ
model_name = "cyberagent/open-calm-3b"   # モデルの名前
peft_name  = "lora-calm-3b"              # 学習用モデル(PEFTモデル)の名前
output_dir = "lora-calm-3b-results"      # 学習結果の出力先

In [2]:
import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer

# モデルの準備
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    load_in_8bit=True,
    device_map="auto",
)

# トークンナイザーの準備
tokenizer = AutoTokenizer.from_pretrained(model_name)

# LoRAモデルの準備
model = PeftModel.from_pretrained(
    model, 
    peft_name, 
    device_map="auto"
)

# 評価モード
model.eval()

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): GPTNeoXForCausalLM(
      (gpt_neox): GPTNeoXModel(
        (embed_in): Embedding(52224, 2560)
        (emb_dropout): Dropout(p=0.0, inplace=False)
        (layers): ModuleList(
          (0-31): 32 x GPTNeoXLayer(
            (input_layernorm): LayerNorm((2560,), eps=1e-05, elementwise_affine=True)
            (post_attention_layernorm): LayerNorm((2560,), eps=1e-05, elementwise_affine=True)
            (post_attention_dropout): Dropout(p=0.0, inplace=False)
            (post_mlp_dropout): Dropout(p=0.0, inplace=False)
            (attention): GPTNeoXAttention(
              (rotary_emb): GPTNeoXRotaryEmbedding()
              (query_key_value): Linear8bitLt(
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=2560, out_features=8, bias=False)
                )
         

In [3]:
def generate_query_prompt(data_point):
    return f"""{data_point['text']}

### Response:
"""

In [4]:
# テキスト生成関数の定義
def generate(instruction,input=None,maxTokens=512):
    # 推論
    prompt = generate_query_prompt({'text':instruction})
    input_ids = tokenizer(prompt, return_tensors="pt", truncation=True).input_ids.cuda()
    outputs = model.generate(
        input_ids=input_ids, 
        max_new_tokens=maxTokens, 
        do_sample=True,
        temperature=0.7, 
        top_p=0.75, 
        top_k=40,         
        no_repeat_ngram_size=2,
    )
    outputs = outputs[0].tolist()

    # EOSトークンにヒットしたらデコード完了
    if tokenizer.eos_token_id in outputs:
        eos_index = outputs.index(tokenizer.eos_token_id)
        decoded = tokenizer.decode(outputs[:eos_index])

        # ラベル内容のみ抽出
        sentinel = "### Response:\n"
        sentinelLoc = decoded.find(sentinel)
        if sentinelLoc >= 0:
            response = decoded[sentinelLoc+len(sentinel):]
            # print(response)
        else:
            print('Warning: Expected prompt template to be emitted.  Ignoring output.')
            response = None
    else:
        print('Warning: no <eos> detected ignoring output')
        response = None
        decoded = None

    return response, prompt, decoded

In [5]:
from datasets import Dataset, DatasetDict

dataset_dict = DatasetDict.load_from_disk('dataset_tokenized')

In [6]:
data_set_test_positive = [d for d in dataset_dict['test'] if d['output'] == 'positive']

In [7]:
len(data_set_test_positive)

1195

In [8]:
# data = dataset_dict["test"][46]
data = data_set_test_positive[16]

response, prompt, decoded = generate(data['text'])
print("---- PROMPT ----")
print(prompt)
print("---- RESPONSE ----")
print(response)
print("---- TARGET ----")
print(data['output'])

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


---- PROMPT ----
スマホがいつ壊れてもおかしくない感じになってきたので昨日買い替えました。AQUOS ZETA SH-03Gです。さくさく動きますね(´∀｀*)

### Response:

---- RESPONSE ----
positive
---- TARGET ----
positive


In [9]:
import tqdm 

results = []
count = 0
for data in tqdm.tqdm(dataset_dict["test"]):
    response, prompt, decoded = generate(data['text'])
    data['pred'] = response
    results.append(data)
    count += 1
    if count > 100:
        break


  0%|          | 0/30417 [00:00<?, ?it/s]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


  0%|          | 1/30417 [00:00<3:15:53,  2.59it/s]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
  0%|          | 2/30417 [00:00<2:10:50,  3.87it/s]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
  0%|          | 3/30417 [00:00<1:49:13,  4.64it/s]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
  0%|          | 4/30417 [00:00<1:39:37,  5.09it/s]The attention mask and the pad token id were not set. A

In [10]:
from sklearn.metrics import classification_report

In [11]:
targets = [d['output'] for d in results]
target_names = list(set(targets))

In [12]:
for d in results:
    if d['pred'] not in target_names:
        d['pred'] = "unrelated"

In [13]:
preds = [d['pred'] for d in results]


In [14]:
print(classification_report(targets, preds, target_names=["positive", "negative", "positive and negative", "neutral", "unrelated"]))

ValueError: Number of classes, 4, does not match size of target_names, 5. Try specifying the labels parameter